In this notebook I will load my model and try to lesion a unit in different layers.

In [3]:
import torch
import torch.nn as nn


In [12]:
import sys
sys.path.append('/home/elaheh_akbari/new/')
sys.path.append('/home/elaheh_akbari/new/sdnn-otherrace')
sys.path.append('/home/elaheh_akbari/new/sdnn-otherrace/models')
sys.path.append('/home/elaheh_akbari/new/sdnn-otherrace/training')
sys.path.append('/home/elaheh_akbari/new/sdnn-otherrace/training/utils')
from utils.helper import Config
from utils.helper import getLayerMapping

In [5]:
# Loading the model from raid
filename = '../../../../raid/elaheh_akbari/face_otherrace_white_asian/checkpoints/vgg/face_otherrace_white_asian/epoch_261.pth.tar'
# checkpoint = torch.load(filename)

In [6]:
config_file = './configs/vgg/face_dual_whitasia.yaml'
config = Config(config_file=config_file)
model, ckpt_data = config.get_model(pretrained=True, ngpus=1, dataParallel=True, epoch=-1)
    

dir: /raid/shared/datasets/visoin/data_face_white/data_facecar_na/train
dir: /raid/shared/datasets/visoin/data_face_asian/data_AFD_matched/train
3308
Restored from: ../../../../raid/elaheh_akbari/face_otherrace_white_asian/checkpoints/vgg/face_otherrace_white_asian/epoch_261.pth.tar


In [8]:
ckpt_data.keys()

dict_keys(['batch_size', 'learning_rate', 'momentum', 'weight_decay', 'walltime', 'epoch', 'keep_epochs', 'state_dict', 'optimizer', 'scheduler'])

In [34]:
# ckpt_data['state_dict']
model

DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [13]:
layerMap = getLayerMapping(model)

In [15]:
paramWeightAndBiasGroupIndex = 0  # decide about the layer 

In [14]:
def getWeightandBias(model, layerMap, paramWeightAndBiasGroupIndex):
    key = 'ParamWeightAndBiasGroupIndex2ParamWeightAndBiasIndexPair'
    weightAndBiasIndexPair = layerMap[key][paramWeightAndBiasGroupIndex]
    weight_index, bias_index = weightAndBiasIndexPair
    tempParameters = list(model.parameters())
    if weight_index is not None:
        weight = tempParameters[weight_index]
    bias=None
    if bias_index is not None:
        bias = tempParameters[bias_index]
    return weight, bias

In [16]:
weight , bias = getWeightandBias(model, layerMap, paramWeightAndBiasGroupIndex)

In [22]:
print(f'The length of the weight is: {len(weight)}')
print(f'The length of the bias is: {len(bias)}')

The length of the weight is: 64
The length of the bias is: 64


In [23]:
cache = {'W':{}, 'b':{}}
for unit in range(len(weight)):
        cache['W'][unit] = weight[unit].detach().cpu().numpy()
        if bias is not None:
            cache['b'][unit] = bias[unit].detach().cpu().numpy()

In [32]:
cache['W'][12]

array([[[-7.36340731e-02, -2.30240393e-02, -9.43707377e-02],
        [-9.67890248e-02, -1.52930245e-01, -1.95417032e-02],
        [-5.13911359e-02, -1.78879574e-01, -8.80359039e-02]],

       [[-7.06956610e-02,  2.28542387e-02, -1.05799204e-02],
        [-1.42074535e-02, -7.92321712e-02, -9.52006802e-02],
        [-6.23369701e-02,  1.71015708e-04, -4.02077213e-02]],

       [[-1.78099163e-02, -3.52552310e-02,  8.13705176e-02],
        [-7.99140781e-02, -9.80853364e-02,  6.72545433e-02],
        [ 2.57421937e-02, -7.37561360e-02, -1.28399171e-02]]],
      dtype=float32)